In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from importlib import reload

# reload from external changes

import datasets
datasets = reload(datasets)
from datasets import *

import ml_util
ml_util = reload(ml_util)
from ml_util import *

import cnn
cnn = reload(cnn)
from cnn import *

import adversarial_generation
adversarial_generation = reload(adversarial_generation)
from adversarial_generation import *

import evaluation_util
evaluation_util = reload(evaluation_util)
from evaluation_util import EvaluationUtil

Util.configure(5000)

In [2]:
#dataset_info = DatasetInfo.CelebA128Gender
dataset_info = DatasetInfo.LSUN128

if dataset_info == DatasetInfo.CelebA128Gender:
    trainer_name = "celeba-128"
    ds = CelebAData(20)
elif dataset_info == DatasetInfo.LSUN128:
    trainer_name = "lsun-128"
    ds = LSUNData()
else:
    raise RuntimeError()
class_weights = [0.5] * 2

In [ ]:
# get some random training images
images, labels = next(iter(ds.get_train_loader(batch_size=8)))
Util.imshow_tensors(images, captions=ds.prediction_indices_to_printed_classes(labels), nrow=8)

In [ ]:
filenames = "oneepoch plain conventional robust both".split(" ")
def load_classifier(weights_filename: str):
    c = Trainer(trainer_name, ds.get_train_loader, ds.get_test_loader)
    c.restore_params_from_disk(weights_filename)
    return c
classifiers = [load_classifier(f"new_classifiers/{filename}_{dataset_info.name}.bin") for filename in filenames]

### A few predictions

In [ ]:
EvaluationUtil.show_some_predictions(classifiers, ds)

### Evaluation: accuracy on the validation set

In [ ]:
EvaluationUtil.evaluate_accuracy(classifiers, ds, 40)

### Evaluation: conventional robustness on the validation set

In [ ]:
EvaluationUtil.evaluate_conventional_robustness(classifiers, ds, 4,
                                                l_2_bounds=[0.0025, 0.005], l_inf_bounds=[0.005, 0.01])

In [ ]:
EvaluationUtil.evaluate_conventional_adversarial_severity(classifiers, ds, 4, OTHER_L2_UPPER_BOUND, OTHER_LINF_UPPER_BOUND)

### Evaluation: optimize class activations

In [ ]:
if dataset_info == DatasetInfo.CelebA128Gender:
    EvaluationUtil.generate_images_with_classifier(classifiers, ds, 2, 0.05, "l_inf", 0.3)
elif dataset_info == DatasetInfo.LSUN128:
    EvaluationUtil.generate_images_with_classifier(classifiers, ds, 2, 0.1, "scaled_l_2", 0.6)